In [73]:
import numpy as np
import pandas as pd 
from transformers import TFBertForSequenceClassification
from transformers import BertTokenizer
import tensorflow as tf

# Correct the path to point directly to the directory containing the model files
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    # Load and compile your model here
    model = TFBertForSequenceClassification.from_pretrained('/kaggle/input/ai-detector-shs/transformers/version-1/1/AI-detector/Model')
    tokenizer = BertTokenizer.from_pretrained('/kaggle/input/ai-detector-shs/transformers/version-1/1/AI-detector/Tokenizer')

Some layers from the model checkpoint at /kaggle/input/ai-detector-shs/transformers/version-1/1/AI-detector/Model were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /kaggle/input/ai-detector-shs/transformers/version-1/1/AI-detector/Model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for pre

In [74]:
test_essay = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')

In [75]:
test_encodings = tokenizer(test_essay['text'].tolist(), truncation=True, padding=True, max_length=512)

In [76]:
import tensorflow as tf
# Create the TensorFlow dataset only for these 3 entries

input_ids_tensor = tf.constant(test_encodings['input_ids'])
attention_mask_tensor = tf.constant(test_encodings['attention_mask'])
token_type_ids_tensor = tf.constant(test_encodings['token_type_ids'])

# Create dataset
test_dataset = tf.data.Dataset.from_tensors({
    'input_ids': input_ids_tensor,
    'attention_mask': attention_mask_tensor,
    'token_type_ids': token_type_ids_tensor
})


# Make predictions using the model
predictions = model.predict(test_dataset)

# Convert logits to probabilities using softmax
prediction_probabilities = tf.nn.softmax(predictions.logits, axis=-1).numpy()

# Extract the probability of the positive class
positive_class_probs = prediction_probabilities[:, 1]

1/1 [==============================] - 0s 394ms/step


In [77]:
results_df = pd.DataFrame({
    'id': test['id'],
    'generated': positive_class_probs
})

# Save the results to a CSV file
results_df.to_csv('submission.csv', index=False)

In [78]:
pd.read_csv('/kaggle/working/submission.csv')

,id,generated
0,0000aaaa,0.134147
1,1111bbbb,0.720276
2,2222cccc,0.142424
